In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder.appName("churn").getOrCreate()

/Users/schanlar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
24/01/19 00:54:41 WARN Utils: Your hostname, Savvass-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.193.99 instead (on interface en0)
24/01/19 00:54:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/19 00:55:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Data

Create a machine learning model that will help predict which customers will churn (stop buying a service) so that a marketing agency can correctly assign the customers most at risk to churn an account manager.

Meaning of data:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company

In [2]:
data = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [3]:
data.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.count()

900

In [6]:
data.describe().show()

24/01/19 00:55:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

We will keep the numerical columns. We will exclude the Account Manager column because because the agency mentioned its randomly assigned, thus conveying no information.

In [7]:
final_dataset = data.select(
    [
        "Age",
        "Total_Purchase",
        "Years",
        "Num_Sites",
        "Churn"
    ]
)

In [8]:
final_dataset.show(5)

+----+--------------+-----+---------+-----+
| Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
|42.0|       11066.8| 7.22|      8.0|    1|
|41.0|      11916.22|  6.5|     11.0|    1|
|38.0|      12884.75| 6.67|     12.0|    1|
|42.0|       8010.76| 6.71|     10.0|    1|
|37.0|       9191.58| 5.56|      9.0|    1|
+----+--------------+-----+---------+-----+
only showing top 5 rows



In [9]:
assembler = VectorAssembler(
    inputCols=[
        "Age",
        "Total_Purchase",
        "Years",
        "Num_Sites",
    ],
    outputCol="features",
)

In [10]:
output = assembler.transform(final_dataset)

In [11]:
final_dataset = output.select('features','Churn')

## Create a Logistic Regression Model

In [12]:
logreg = LogisticRegression(featuresCol='features', labelCol='Churn')

In [13]:
train_set, test_set = final_dataset.randomSplit([0.7, 0.3])

In [14]:
fitted_model = logreg.fit(train_set)

24/01/19 00:55:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/19 00:55:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [15]:
training_summary = fitted_model.summary

In [16]:
training_summary.predictions.describe().show()

+-------+------------------+------------------+
|summary|             Churn|        prediction|
+-------+------------------+------------------+
|  count|               649|               649|
|   mean|0.1679506933744222|0.1263482280431433|
| stddev|0.3741108273564825|0.3324976684696456|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



## Model Evaluation

In [17]:
results = fitted_model.transform(test_set)

In [18]:
results.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,5.4...|    1|[0.68710269512529...|[0.66532209723654...|       0.0|
|[28.0,9090.43,5.7...|    0|[1.58688759786459...|[0.83017775918952...|       0.0|
|[28.0,11245.38,6....|    0|[3.40964441075799...|[0.96800459113878...|       0.0|
|[29.0,9378.24,4.9...|    0|[4.45069374006238...|[0.98846416075126...|       0.0|
|[29.0,11274.46,4....|    0|[4.78844072835305...|[0.99174331160053...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [19]:
results.select("Churn", "prediction").show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
model_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [22]:
auc = model_eval.evaluate(results)

In [23]:
auc

0.7394308943089432

### Predict on brand new unlabeled data

In [24]:
# Re-train on entire dataset
final_model = logreg.fit(final_dataset)

In [25]:
new_data = spark.read.csv('new_customers.csv', header=True, inferSchema=True)
new_data.show(5)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [26]:
new_customers = assembler.transform(new_data)

In [27]:
final_results = final_model.transform(new_customers)

In [28]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Based on our model, we should assign Acocunt Managers to **Cannon-Benson**, **Barron-Robertson**, **Sexton-Golden**, and **Parks-Robbins**!